In [1]:
import pandas as pd
import numpy as np

In [2]:
BD = "bookings_data"
B = "bookings"
CD = "customer_data"
HD = "hotels_data"
PD = "payments_data"
SS = "sample_submission_5"
TD = "train_data"

base_path = r"../data/"

def load_from_csv(filename):
    file_path = base_path + filename + ".csv"
    return pd.read_csv(file_path)

In [3]:
bookings_data = load_from_csv(BD)
bookings = load_from_csv(B)
customer_data = load_from_csv(CD)
hotels_data = load_from_csv(HD)
payments_data = load_from_csv(PD)
sample_submission = load_from_csv(SS)
train_data = load_from_csv(TD)

bookings['booking_create_timestamp'] = pd.to_datetime(bookings['booking_create_timestamp'])
bookings['booking_approved_at'] = pd.to_datetime(bookings['booking_approved_at'])
bookings['booking_checkin_customer_date'] = pd.to_datetime(bookings['booking_checkin_customer_date'])
bookings_data['booking_expiry_date'] = pd.to_datetime(bookings_data['booking_expiry_date'])

In [4]:
def get_hash(val_list):
    hashmap = {}
    for i,val in enumerate(val_list):
        hashmap[val] = i
    return hashmap

In [5]:
unique_ids = payments_data['booking_id'].unique()
hash_val = get_hash(unique_ids)
counts = np.zeros(unique_ids.shape)
total_installments = np.zeros(unique_ids.shape)
value_sum = np.zeros(unique_ids.shape)
for i in range(payments_data.shape[0]):
    booking_id = payments_data.at[i, 'booking_id']
    installments = payments_data.at[i, 'payment_installments']
    payment_value = payments_data.at[i, 'payment_value']
    counts[hash_val[booking_id]] += 1
    total_installments[hash_val[booking_id]] += installments
    value_sum[hash_val[booking_id]] += payment_value
payments_data_modified = []
for booking_id in unique_ids:
    payments_made =  counts[hash_val[booking_id]]
    payments_installments = total_installments[hash_val[booking_id]]
    payments_value = value_sum[hash_val[booking_id]]
    payments_data_modified.append([booking_id, payments_made, payments_installments, payments_value])
payments_data_modified = pd.DataFrame(data=payments_data_modified, columns=['booking_id', 'payments_made', 'payments_installments', 'payments_value'])
payments_data_modified.describe(include='all')

,booking_id,payments_made,payments_installments,payments_value
count,99440,99440.000000,99440.000000,99440.000000
unique,99440,NaN,NaN,NaN
top,6f3fe1789b1e8b2acac839d17b81ef22,NaN,NaN,NaN
freq,1,NaN,NaN,NaN
mean,NaN,1.044710,2.980923,160.990267
std,NaN,0.381166,2.741810,221.951257
min,NaN,1.000000,0.000000,0.000000
25%,NaN,1.000000,1.000000,62.010000
50%,NaN,1.000000,2.000000,105.290000
75%,NaN,1.000000,4.000000,176.970000


In [6]:
# Missing: dates besides created
unique_booking_status = bookings['booking_status'].unique()
# status_hash = get_hash(unique_booking_status)
bookings_modified = []
for i in range(bookings.shape[0]):
    booking_id = bookings.at[i, 'booking_id']
    customer_id = bookings.at[i, 'customer_id']
    booking_status_num = bookings.at[i, 'booking_status']
    create_date = bookings.at[i, 'booking_create_timestamp']
    approved_date = bookings.at[i, 'booking_approved_at']
    checkin_date = bookings.at[i, 'booking_checkin_customer_date']
    approval_time = (approved_date-create_date).total_seconds()/60
    checkin_time = (checkin_date-create_date).total_seconds()/1440
    bookings_modified.append([booking_id, customer_id, booking_status_num, create_date, approval_time, checkin_time])
bookings_modified = pd.DataFrame(data=bookings_modified, columns=[
    'booking_id', 'customer_id', 'booking_status', 'booking_create_timestamp', 'booking_approval_time', 'booking_checkin_time'])
# bookings_modified.fillna(-1, inplace=True)
bookings_modified.describe(include='all')

/tmp/ipykernel_19100/51974090.py:18: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  bookings_modified.describe(include='all')


,booking_id,customer_id,booking_status,booking_create_timestamp,booking_approval_time,booking_checkin_time
count,99441,99441,99441,99441,99281.000000,96476.000000
unique,99441,99441,8,98875,NaN,NaN
top,c54678b7cc49136f2d6af7e481f51cbd,51297304e76186b10a928d9ef432eb62,completed,2008-04-13 10:31:14,NaN,NaN
freq,1,1,96478,3,NaN,NaN
first,NaN,NaN,NaN,2006-09-07 20:58:19,NaN,NaN
last,NaN,NaN,NaN,2008-10-19 17:13:18,NaN,NaN
mean,NaN,NaN,NaN,NaN,625.145658,753.522138
std,NaN,NaN,NaN,NaN,1562.280218,572.791792
min,NaN,NaN,NaN,NaN,0.000000,32.004861
25%,NaN,NaN,NaN,NaN,12.900000,405.984201


In [7]:
unique_ids = bookings_data['booking_id'].unique()
hash_val = get_hash(unique_ids)
unique_agents = bookings_data['seller_agent_id'].unique()
# agent_hash = get_hash(unique_agents)
counts = np.zeros(unique_ids.shape)
hotel_ids = np.zeros(unique_ids.shape).tolist()
seller_agent_ids = np.zeros(unique_ids.shape).tolist()
booking_expiry_dates = np.zeros(unique_ids.shape).tolist()
prices = np.zeros(unique_ids.shape).tolist()
agent_feess = np.zeros(unique_ids.shape).tolist()
for i in range(bookings_data.shape[0]):
    booking_id = bookings_data.at[i, 'booking_id']
    counts[hash_val[booking_id]] += 1
    hotel_ids[hash_val[booking_id]] = bookings_data.at[i, 'hotel_id']
    seller_agent_ids[hash_val[booking_id]] = bookings_data.at[i, 'seller_agent_id']
    booking_expiry_dates[hash_val[booking_id]] = bookings_data.at[i, 'booking_expiry_date']
    prices[hash_val[booking_id]] = bookings_data.at[i, 'price']
    agent_feess[hash_val[booking_id]] = bookings_data.at[i, 'agent_fees']
bookings_data_modified = []
for booking_id in unique_ids:
    sub_requests = counts[hash_val[booking_id]]
    hotel_id = hotel_ids[hash_val[booking_id]]
    seller_agent_id = seller_agent_ids[hash_val[booking_id]]
    booking_expiry_date = booking_expiry_dates[hash_val[booking_id]]
    price = prices[hash_val[booking_id]]
    agent_fees = agent_feess[hash_val[booking_id]]
    bookings_data_modified.append([booking_id, sub_requests, hotel_id, seller_agent_id, booking_expiry_date, price, agent_fees])
bookings_data_modified =  pd.DataFrame(data=bookings_data_modified, columns=[
    'booking_id', 'sub_requests', 'hotel_id', 'seller_agent_id', 'booking_expiry_date', 'price', 'agent_fees'
])
bookings_data_modified.describe(include='all')

/tmp/ipykernel_19100/3407971356.py:31: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  bookings_data_modified.describe(include='all')


,booking_id,sub_requests,hotel_id,seller_agent_id,booking_expiry_date,price,agent_fees
count,98666,98666.000000,98666,98666,98666,98666.000000,98666.000000
unique,98666,NaN,31862,3084,93009,NaN,NaN
top,242fe8c5a6d1ba2dd792cb1621400010,NaN,856965c36a24e339b605899a4788cb24,19b47992c3666cc44a7e94c06560211a,2018-06-11 03:31:04,NaN,NaN
freq,1,NaN,443,1835,6,NaN,NaN
first,NaN,NaN,NaN,NaN,2016-09-19 00:15:34,NaN,NaN
last,NaN,NaN,NaN,NaN,2020-04-09 22:35:08,NaN,NaN
mean,NaN,1.141731,NaN,NaN,NaN,125.883144,20.179411
std,NaN,0.538452,NaN,NaN,NaN,191.166764,15.855944
min,NaN,1.000000,NaN,NaN,NaN,0.850000,0.000000
25%,NaN,1.000000,NaN,NaN,NaN,41.492500,13.280000


In [8]:
train_data_full = pd.merge(left=train_data, right=bookings_modified, how='left', on='booking_id')
train_data_full = pd.merge(left=train_data_full, right=bookings_data_modified, how='left', on='booking_id')
train_data_full = pd.merge(left=train_data_full, right=customer_data, how='left', on='customer_id')
train_data_full = pd.merge(left=train_data_full, right=hotels_data, how='left', on='hotel_id')
train_data_full = pd.merge(left=train_data_full, right=payments_data_modified, how='left', on='booking_id')
# train_data_full.describe(include='all')
print(train_data_full.shape[0])
booking_expiry = []
unique_ids = train_data_full['booking_id'].unique()
id_hash = get_hash(unique_ids)
expiry_times = np.zeros(unique_ids.shape)
for i in range(train_data_full.shape[0]):
    booking_id = train_data_full.at[i, 'booking_id']
    expiry_date = train_data_full.at[i, 'booking_expiry_date']
    create_date = train_data_full.at[i, 'booking_create_timestamp']
    expiry_time = (expiry_date-create_date).total_seconds()/1440
    expiry_times[id_hash[booking_id]] = expiry_time
for booking_id in unique_ids:
    expiry_time = expiry_times[id_hash[booking_id]]
    booking_expiry.append([booking_id, expiry_time])
booking_expiry = pd.DataFrame(data=booking_expiry, columns=['booking_id', 'booking_expiry_time'])
print(booking_expiry.shape[0])
train_data_full = pd.merge(left=train_data_full, right=booking_expiry, how='left', on='booking_id')
train_data_full.drop(labels=['booking_create_timestamp', 'booking_expiry_date'], axis=1, inplace=True)

50000
49868


In [9]:
test_data = sample_submission['booking_id']
test_data = pd.merge(left=test_data, right=bookings_modified, how='left', on='booking_id')
test_data = pd.merge(left=test_data, right=bookings_data_modified, how='left', on='booking_id')
test_data = pd.merge(left=test_data, right=customer_data, how='left', on='customer_id')
test_data = pd.merge(left=test_data, right=hotels_data, how='left', on='hotel_id')
test_data = pd.merge(left=test_data, right=payments_data_modified, how='left', on='booking_id')
# test_data.describe(include='all')
print(test_data.shape[0])
booking_expiry = []
unique_ids = test_data['booking_id'].unique()
id_hash = get_hash(unique_ids)
expiry_times = np.zeros(unique_ids.shape)
for i in range(test_data.shape[0]):
    booking_id = test_data.at[i, 'booking_id']
    expiry_date = test_data.at[i, 'booking_expiry_date']
    create_date = test_data.at[i, 'booking_create_timestamp']
    expiry_time = (expiry_date-create_date).total_seconds()/1440
    expiry_times[id_hash[booking_id]] = expiry_time
for booking_id in unique_ids:
    expiry_time = expiry_times[id_hash[booking_id]]
    booking_expiry.append([booking_id, expiry_time])
booking_expiry = pd.DataFrame(data=booking_expiry, columns=['booking_id', 'booking_expiry_time'])
print(booking_expiry.shape[0])
test_data = pd.merge(left=test_data, right=booking_expiry, how='left', on='booking_id')
test_data.drop(labels=['booking_create_timestamp', 'booking_expiry_date'], axis=1, inplace=True)

49079
49079


In [10]:
from sklearn.model_selection import train_test_split
train_labels = train_data_full['rating_score']
train_data_full.drop(labels=['rating_score', 'booking_id', 'customer_id'], axis=1, inplace=True)
X_train, X_val, Y_train, Y_val = train_test_split(train_data_full, train_labels, test_size=0.2)
test_ids = test_data['booking_id']
test_data.drop(labels=['booking_id', 'customer_id'], axis=1, inplace=True)

In [11]:
for column in X_train.columns:
    print(column)
    print(X_train[column].unique().shape[0])
    print('')

booking_status
8

booking_approval_time
15986

booking_checkin_time
38319

sub_requests
15

hotel_id
17884

seller_agent_id
2557

price
3898

agent_fees
4942

customer_unique_id
39259

country
9

hotel_category
74

hotel_name_length
63

hotel_description_length
2560

hotel_photos_qty
19

payments_made
17

payments_installments
24

payments_value
17482

booking_expiry_time
23039



In [12]:
X_train.dtypes

booking_status               object
booking_approval_time       float64
booking_checkin_time        float64
sub_requests                float64
hotel_id                     object
seller_agent_id              object
price                       float64
agent_fees                  float64
customer_unique_id           object
country                      object
hotel_category              float64
hotel_name_length           float64
hotel_description_length    float64
hotel_photos_qty            float64
payments_made               float64
payments_installments       float64
payments_value              float64
booking_expiry_time         float64
dtype: object

In [13]:
onehotcols = ['booking_status', 'country']
ordinalcols = ['hotel_id', 'seller_agent_id', 'customer_unique_id']
numcols = X_train.columns[X_train.dtypes == 'float64'].tolist()
numcols

['booking_approval_time',
 'booking_checkin_time',
 'sub_requests',
 'price',
 'agent_fees',
 'hotel_category',
 'hotel_name_length',
 'hotel_description_length',
 'hotel_photos_qty',
 'payments_made',
 'payments_installments',
 'payments_value',
 'booking_expiry_time']

In [14]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler
from sklearn.metrics import mean_squared_error

In [19]:
num_transformer = Pipeline(steps=[
    ('impute', SimpleImputer(strategy='constant', fill_value=-1)),
    ('scale', StandardScaler(with_mean=False))
])

oh_transformer = Pipeline(steps=[
    ('impute', SimpleImputer(strategy='constant', fill_value='Missing')),
    ('ohenc', OneHotEncoder(handle_unknown='infrequent_if_exist')),
    ('scale', StandardScaler(with_mean=False))
])

ord_transformer = Pipeline(steps=[
    ('impute', SimpleImputer(strategy='constant', fill_value='Missing')),
    ('ordenc', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)),
    ('scale', StandardScaler(with_mean=False))
])

# scaler = StandardScaler()

preprocess = ColumnTransformer(
    transformers=[
        ('num', num_transformer, numcols),
        ('ord', ord_transformer, ordinalcols),
        ('oh', oh_transformer, onehotcols)
    ]
)

In [30]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, HistGradientBoostingRegressor
from xgboost import XGBRegressor

# model = RandomForestRegressor(n_estimators=1000, max_features=0.5)
# model = GradientBoostingRegressor(n_estimators=1000)
model = HistGradientBoostingRegressor(max_iter=100, max_depth=9)

In [17]:
def trim_pred(preds):
    for i in range(preds.shape[0]):
        if preds[i] > 5:
            preds[i] = 5
        elif preds[i] < 1:
            preds[i] = 1
    return preds

In [31]:
model_pipeline = Pipeline(steps=[
    ('preprocess', preprocess),
    ('model', model)
])

# model_pipeline.fit(X_train, Y_train)

# preds = trim_pred(model_pipeline.predict(X_val))

# score = mean_squared_error(Y_val, preds)

# score

In [ ]:
from sklearn.model_selection import GridSearchCV

"""
For HistGradientBoostingRegressor, best params: max_depth = 9, max_iter = 100
"""

parameters = {
    'model__max_iter': [100, 200, 500, 1000, 5000],
    'model__max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
}

pipeline_cv = GridSearchCV(model_pipeline, parameters, refit=True)
pipeline_cv.fit(X_train, Y_train)
print(pipeline_cv.best_params_)

In [28]:
preds = trim_pred(pipeline_cv.predict(X_val))

score = mean_squared_error(Y_val, preds)

score

1.3195918847884935

In [21]:
test_preds = trim_pred(model_pipeline.predict(test_data))
test_preds = pd.DataFrame(data=test_preds, columns=['rating_score'])
test_sub = pd.concat([test_ids, test_preds], axis=1)
test_sub.describe(include='all')

,booking_id,rating_score
count,49079,49079.000000
unique,49079,NaN
top,796b98fdf73dbeba33a548910a1c6147,NaN
freq,1,NaN
mean,NaN,4.084705
std,NaN,0.653348
min,NaN,1.000000
25%,NaN,4.055937
50%,NaN,4.336280
75%,NaN,4.425246


In [ ]:
# test_sub.to_csv("../output/RegressionHGB_Scale.csv", index=False)